# Exercise 6

Authors: \
Tuoxing Liu \
Sima Esmaeili \
Shruti Ghargi

# 1 Classification and Regression Tree

In [1]:
import numpy as np
from sklearn.datasets import load_digits
from abc import abstractmethod

## Base Classes

In [2]:
class Node:
    '''
      this class will later get the following attributes
      all nodes:
          features
          responses
      split nodes additionally:
          left
          right
          split_index
          threshold
      leaf nodes additionally
          prediction
    '''
    pass


class Tree:
    '''
      base class for RegressionTree and ClassificationTree
    '''
    def __init__(self, n_min=10):
        '''n_min: minimum required number of instances in leaf nodes
        '''
        self.n_min = n_min 
    
    def predict(self, x):
        ''' return the prediction for the given 1-D feature vector x
        '''
        # first find the leaf containing the 1-D feature vector x
        node = self.root
        while not hasattr(node, "prediction"):
            j = node.split_index
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        # finally, return the leaf's prediction
        return node.prediction
        
    def train(self, features, responses, D_try=None):
        '''
        features: the feature matrix of the training set
        response: the vector of responses
        '''
        N, D = features.shape
        assert(responses.shape[0] == N)

        if D_try is None:
            D_try = int(np.sqrt(D))  # number of features to consider for each split decision
        
        # initialize the root node
        self.root = Node()
        self.root.features = features
        self.root.responses = responses

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            active_indices = self.select_active_indices(D, D_try)
            left, right = self.make_split_node(node, active_indices)
            if left is None:  # no split found
                self.make_leaf_node(node)
            else:
                stack.append(left)
                stack.append(right)
    
    def make_split_node(self, node, indices):
        '''
        node: the node to be split
        indices: a numpy array of length 'D_try', containing the feature 
                         indices to be considered for the present split
                         
        return: None, None -- if no suitable split has been found, or
                left, right -- the children of the split
        '''
        # all responses equal => no improvement possible by any split
        if np.unique(node.responses).shape[0] == 1:
            return None, None
        
        # find best feature j_min (among 'indices') and best threshold t_min for the split
        l_min = float('inf')  # upper bound for the loss, later the loss of the best split
        j_min, t_min = None, None

        for j in indices:
            thresholds = self.find_thresholds(node, j)

            # compute loss for each threshold
            for t in thresholds:
                loss = self.compute_loss_for_split(node, j, t)

                # remember the best split so far 
                # (the condition is never True when loss = float('inf') )
                if loss < l_min:
                    l_min = loss
                    j_min = j
                    t_min = t

        if j_min is None:  # no split found
            return None, None

        # create children for the best split
        left, right = self.make_children(node, j_min, t_min)

        # turn the current 'node' into a split node
        node.split_index = j_min
        node.threshold = t_min
        node.left = left
        node.right = right
        
        # return the children (to be placed on the stack)
        return left, right
    
    def select_active_indices(self, D, D_try):
        ''' return a 1-D array with D_try randomly selected indices from 0...(D-1).
        '''
        return np.random.choice(D, D_try, replace=False)
    
    def find_thresholds(self, node, j):
        ''' return: a 1-D array with all possible thresholds along feature j
        '''
        return np.sort(np.unique(node.features[:, j]))
    
    def make_children(self, node, j, t):
        ''' execute the split in feature j at threshold t
        
            return: left, right -- the children of the split, with features and responses
                                   properly assigned according to the split
        '''
        left = Node()
        right = Node()

        indices_left = node.features[:, j] <= t
        indices_right = ~indices_left
        
        left.features = node.features[indices_left]
        left.responses = node.responses[indices_left]
        
        right.features = node.features[indices_right]
        right.responses = node.responses[indices_right]
        
        return left, right
        
    @abstractmethod
    def make_leaf_node(self, node):
        ''' Turn node into a leaf by computing and setting `node.prediction`
        
            (must be implemented in a subclass)
        '''
        raise NotImplementedError("make_leaf_node() must be implemented in a subclass.")
        
    @abstractmethod
    def compute_loss_for_split(self, node, j, t):
        ''' Return the resulting loss when the data are split along feature j at threshold t.
            If the split is not admissible, return float('inf').
        
            (must be implemented in a subclass)
        '''
        raise NotImplementedError("compute_loss_for_split() must be implemented in a subclass.")

## Regression Tree

In [3]:
class RegressionTree(Tree):
    def __init__(self, n_min=10):
        super(RegressionTree, self).__init__(n_min)
        
    def compute_loss_for_split(self, node, j, t):
        # return the loss if we would split the instance along feature j at threshold t
        # or float('inf') if there is no feasible split
        left_indices = node.features[:, j] <= t
        right_indices = ~left_indices

        if np.sum(left_indices) < self.n_min or np.sum(right_indices) < self.n_min:
            return float('inf')

        left_responses = node.responses[left_indices]
        right_responses = node.responses[right_indices]

        left_loss = np.mean((left_responses - np.mean(left_responses))**2)
        right_loss = np.mean((right_responses - np.mean(right_responses))**2)

        return left_loss + right_loss
    
    def make_leaf_node(self, node):
        # turn node into a leaf node by computing `node.prediction`
        # (note: the prediction of a regression tree is a real number)
        node.prediction = np.mean(node.responses)
        
    def compute_loss_for_split(self, node, j, t):
        # return the loss if we would split the instance along feature j at threshold t
        # or float('inf') if there is no feasible split
        left_indices = node.features[:, j] <= t
        right_indices = ~left_indices

        if np.sum(left_indices) < self.n_min or np.sum(right_indices) < self.n_min:
            return float('inf')

        left_responses = node.responses[left_indices]
        right_responses = node.responses[right_indices]

        left_loss = np.mean((left_responses - np.mean(left_responses))**2)
        right_loss = np.mean((right_responses - np.mean(right_responses))**2)

        return left_loss + right_loss
    
    def make_leaf_node(self, node):
        # turn node into a leaf node by computing `node.prediction`
        # (note: the prediction of a regression tree is a real number)
        node.prediction = np.mean(node.responses)

## Classification Tree

In [4]:
class ClassificationTree(Tree):
    '''implement classification tree so that it can handle arbitrary many classes
    '''
    
    def __init__(self, classes, n_min=10):
        ''' classes: a 1-D array with the permitted class labels
            n_min: minimum required number of instances in leaf nodes
        '''
        super(ClassificationTree, self).__init__(n_min)
        self.classes = classes
        
    def compute_loss_for_split(self, node, j, t):
        # return the loss if we would split the instance along feature j at threshold t
        # or float('inf') if there is no feasible split
        left_indices = node.features[:, j] <= t
        right_indices = ~left_indices

        if np.sum(left_indices) < self.n_min or np.sum(right_indices) < self.n_min:
            return float('inf')

        left_responses = node.responses[left_indices]
        right_responses = node.responses[right_indices]

        left_loss = self.compute_loss(left_responses)
        right_loss = self.compute_loss(right_responses)

        return (np.sum(left_indices) * left_loss + np.sum(right_indices) * right_loss) / np.sum(left_indices + right_indices)
    
    def compute_loss(self, responses):
        class_counts = np.bincount(responses, minlength=len(self.classes))
        class_probabilities = class_counts / np.sum(class_counts)

        return self.compute_entropy(class_probabilities)
    
    def compute_entropy(self, probabilities):
        probabilities = probabilities[probabilities > 0]
        return -np.sum(probabilities * np.log2(probabilities))
    
    def make_leaf_node(self, node):
        # turn node into a leaf node by computing `node.prediction`
        # (note: the prediction of a classification tree is a class label)
        class_counts = np.bincount(node.responses, minlength=len(self.classes))
        node.prediction = np.argmax(class_counts)
        

## Evaluation of Regression and Classification Tree

In [5]:
# read and prepare the digits data and extract 3s and 9s
digits = load_digits()
print(digits.data.shape, digits.target.shape)

instances = (digits.target == 3) | (digits.target == 9)
features = digits.data[instances, :]
labels = digits.target[instances]

# for regression, we use labels +1 and -1
responses = np.array([1 if l == 3 else -1 for l in labels])

assert(features.shape[0] == labels.shape[0] == responses.shape[0])

(1797, 64) (1797,)


For Regression Tree:

In [6]:
from sklearn.model_selection import KFold

# Create an instance of RegressionTree
regression_tree = RegressionTree()

# Perform 5-fold cross-validation
kf = KFold(n_splits=5)
mse_scores = []

for train_index, test_index in kf.split(features):
    train_features, test_features = features[train_index], features[test_index]
    train_responses, test_responses = responses[train_index], responses[test_index]

    # Train the regression tree
    regression_tree.train(train_features, train_responses)

    # Predict the responses for the test set
    predictions = [regression_tree.predict(x) for x in test_features]

    # Calculate the mean squared error (MSE)
    mse = np.mean((test_responses - predictions) ** 2)
    mse_scores.append(mse)

# Calculate and print the mean MSE score
mean_mse = np.mean(mse_scores)
print("Regression Tree Mean Accuracy:", mean_mse)


Regression Tree Mean Accuracy: 0.38923872309061275


For Classification Tree:

In [7]:
from sklearn.model_selection import StratifiedKFold

# Create an instance of ClassificationTree
classification_tree = ClassificationTree(classes=np.unique(labels))

# Perform 5-fold cross-validation
skf = StratifiedKFold(n_splits=5)
accuracy_scores = []

for train_index, test_index in skf.split(features, labels):
    train_features, test_features = features[train_index], features[test_index]
    train_labels, test_labels = labels[train_index], labels[test_index]

    # Train the classification tree
    classification_tree.train(train_features, train_labels)

    # Predict the labels for the test set
    predictions = [classification_tree.predict(x) for x in test_features]

    # Calculate the accuracy
    accuracy = np.mean(predictions == test_labels)
    accuracy_scores.append(accuracy)

# Calculate and print the mean accuracy score
mean_accuracy = np.mean(accuracy_scores)
print("Classification Tree Mean Accuracy:", mean_accuracy)


Classification Tree Mean Accuracy: 0.8762176560121766
